In [ ]:
import pandas as pd
import numpy as np
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show, output_file, vplot, save
output_notebook()
#output_file('sensex2.html')

### Read Data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/nrasiwas/randomfiles/master/data/SENSEX.csv', error_bad_lines=False)
df.tail()

### Analyze Data 

In [ ]:
#startord = datetime.date(2000, 1, 1).toordinal()
startord = datetime.date(1979, 4, 3).toordinal()

endord = datetime.date(2016, 6, 10).toordinal()
dates = []
years = [2,5,10]
N = endord - startord + 1
D = 1
out = np.zeros((N / D, len(years)))
outdf = None
for sno, y in enumerate(years):
    for isno, i in enumerate(range(0,N,D)):
        try:
            startdate = datetime.datetime.fromordinal(startord + i)
            if sno == 0:
                dates.append(startdate)
            starttext = startdate.strftime('%d-%B-%Y').lstrip("0").replace(" 0", " ")
            enddate = startdate.replace(year = startdate.year + y)
            endtext = enddate.strftime('%d-%B-%Y').lstrip("0").replace(" 0", " ")
            startvalue = df.loc[starttext].Low
            endvalue = df.loc[endtext].Low
            rr = (np.power(endvalue / startvalue, 1 / float(y)) - 1) * 100
        except:
            rr = np.NaN
        out[isno, sno] = rr

In [ ]:
outdf = pd.DataFrame(out, index=dates).interpolate()
outdf.columns = ['{} Year'.format(i) for i in years]
for sno, y in enumerate(years):
    outdf['{} Year'.format(y)][-(365 * y):] = np.NaN
outdf.index = dates
outdf.describe()
sampled = outdf.iloc[::7]
print sampled.count()

### Visualize Data 

In [ ]:
from bokeh.palettes import Spectral11
from bokeh.models import HoverTool
numlines=len(sampled.columns)
mypalette=Spectral11[0:numlines]
p = {}
#"pan,wheel_zoom,reset,resize,save,crosshair,hover", 

for sno, name in enumerate(sampled):
    mn = sampled[name].mean()
    st = sampled[name].std()
    d1 = sampled.index.values[0]
    d2 = sampled.index.values[-1]
    p[sno] = figure(tools=['pan,reset,resize,wheel_zoom,crosshair,save'], x_axis_type="datetime", x_axis_label='date', y_axis_label='%', title= name + ' returns (%)', width=1000, height=600)
    g_mean = p[sno].line([d1, d2], [mn, mn], line_width=2, line_color = mypalette[sno])
    p[sno].quad(left=[d1], right=[d2], top=[mn+2*st], bottom=[mn-2*st], color="#B369DE", fill_alpha=0.1, line_alpha=0)
    p[sno].quad(left=[d1], right=[d2], top=[mn+st], bottom=[mn-st], color="#B3DE69", fill_alpha=0.3, line_alpha=0)
    g_line = p[sno].line(sampled.index.values, sampled[name].values, line_width=2, line_color = mypalette[sno])
    hover = HoverTool(
        renderers=[g_line, g_mean],
        tooltips=[("Interest", "@y{2.2}%")]
    )
    p[sno].add_tools(hover)

vp = vplot(p[0], p[1], p[2])
show(vp)